In [2]:
import mysql.connector as conn
import pandas as pd
db = conn.connect(
    host='localhost',
    user='root',
    password='123456789zxcvbnm',
    database = 'shop',
)

# 创建游标对象，以便执行 SQL 语句
cursor = db.cursor(buffered=True)

In [8]:
select_all = "SELECT * FROM products"
cursor.execute(select_all)

# 获取查询结果
res = cursor.fetchall()

# 将结果转换
df_data1 = pd.DataFrame(res, columns=cursor.column_names)
df_data1

,product_id,product_name,product_type,sale_price,purchase_price,regist_date
0,0001,T恤衫,衣服,1000,500.0,2009-09-20
1,0002,打孔器,办公用品,500,320.0,2009-09-11
2,0003,运动T恤,衣服,4000,2800.0,None
3,0004,菜刀,厨房用具,3000,2800.0,2009-09-20
4,0005,高压锅,厨房用具,6800,5000.0,2009-01-15
5,0006,叉子,厨房用具,500,NaN,2009-09-20
6,0007,擦菜板,厨房用具,880,790.0,2008-04-28
7,0008,圆珠笔,办公用品,100,NaN,2009-11-11


##### 视图

In [6]:

### 基于单表视图
creat_view = '''CREATE VIEW productsum1 (product_type, cnt_product)
                AS
                SELECT product_type, COUNT(*)
                FROM products
                GROUP BY product_type; '''
cursor.execute(creat_view)
db.commit()

# 查询视图中的结果
query_view = '''SELECT * FROM productsum1'''
cursor.execute(query_view)

res = cursor.fetchall()
print(res)

df_view = pd.DataFrame(res, columns=cursor.column_names)
df_view


[('衣服', 2), ('办公用品', 2), ('厨房用具', 4)]


,product_type,cnt_product
0,衣服,2
1,办公用品,2
2,厨房用具,4


In [7]:
# 创建一张表
creat_table = '''CREATE TABLE shop_product
                 (shop_id    CHAR(4)       NOT NULL,
                  shop_name  VARCHAR(200)  NOT NULL,
                  product_id CHAR(4)       NOT NULL,
                  quantity   INTEGER       NOT NULL,
                  PRIMARY KEY (shop_id, product_id)) '''
cursor.execute(creat_table)
db.commit()

# 插入数据

insert_data1 = '''INSERT INTO shop_product (shop_id, shop_name, product_id, quantity) VALUES
                  ('000A',	'东京',		'0001',	30),
                  ('000A',	'东京',		'0002',	50),
                  ('000A',	'东京',		'0003',	15),
                  ('000B',	'名古屋',	'0002',	30),
                  ('000B',	'名古屋',	'0003',	120),
                  ('000B',	'名古屋',	'0004',	20),
                  ('000B',	'名古屋',	'0006',	10),
                  ('000B',	'名古屋',	'0007',	40),
                  ('000C',	'大阪',		'0003',	20),
                  ('000C',	'大阪',		'0006',	90),
                  ('000C',	'大阪',		'0004',	50),
                  ('000C',	'大阪',		'0007',	70),
                  ('000D',	'福冈',		'0001',	100); '''
cursor.execute(insert_data1)
db.commit()


select_all = "SELECT * FROM shop_product"
cursor.execute(select_all)

res = cursor.fetchall()
df_data = pd.DataFrame(res, columns=cursor.column_names)
df_data

,shop_id,shop_name,product_id,quantity
0,000A,东京,0001,30
1,000A,东京,0002,50
2,000A,东京,0003,15
3,000B,名古屋,0002,30
4,000B,名古屋,0003,120
5,000B,名古屋,0004,20
6,000B,名古屋,0006,10
7,000B,名古屋,0007,40
8,000C,大阪,0003,20
9,000C,大阪,0004,50


In [9]:
df_data1.columns

Index(['product_id', 'product_name', 'product_type', 'sale_price',
       'purchase_price', 'regist_date'],
      dtype='object')

In [10]:
df_data.columns

Index(['shop_id', 'shop_name', 'product_id', 'quantity'], dtype='object')

In [12]:
### 基于多表的视图
'''
创建一个视图，有三列，这个视图表示的是：
从两个表中选择的三个列，其中的 product_id 是相等的。
'''

creat_view_multigraph = '''CREATE VIEW view_shop_product(product_type, sale_price, shop_name)
                           AS
                           SELECT product_type, sale_price, shop_name
                           FROM products, shop_product
                           WHERE products.product_id = shop_product.product_id; '''

cursor.execute(creat_view_multigraph)
db.commit()

query_view_multigraph = '''SELECT * FROM view_shop_product'''
cursor.execute(query_view_multigraph)

res = cursor.fetchall()
print(res)

df_view_multigraph = pd.DataFrame(res, columns=cursor.column_names)
df_view_multigraph

[('衣服', 1000, '东京'), ('办公用品', 500, '东京'), ('衣服', 4000, '东京'), ('办公用品', 500, '名古屋'), ('衣服', 4000, '名古屋'), ('厨房用具', 3000, '名古屋'), ('厨房用具', 500, '名古屋'), ('厨房用具', 880, '名古屋'), ('衣服', 4000, '大阪'), ('厨房用具', 3000, '大阪'), ('厨房用具', 500, '大阪'), ('厨房用具', 880, '大阪'), ('衣服', 1000, '福冈')]


,product_type,sale_price,shop_name
0,衣服,1000,东京
1,办公用品,500,东京
2,衣服,4000,东京
3,办公用品,500,名古屋
4,衣服,4000,名古屋
5,厨房用具,3000,名古屋
6,厨房用具,500,名古屋
7,厨房用具,880,名古屋
8,衣服,4000,大阪
9,厨房用具,3000,大阪


##### 修改视图

In [13]:
# 修改 productsum1 
modify_view = '''ALTER VIEW productsum1
                AS
                SELECT product_type, sale_price
                FROM products
                WHERE regist_date > '2009-09-11'; '''
cursor.execute(modify_view)
# db.commit() 视图定义是属于数据库的元数据操作，不是对表数据的修改。 


# 查询视图中的结果
query_view = '''SELECT * FROM productsum1'''
cursor.execute(query_view)

res = cursor.fetchall()
print(res)

df_view1 = pd.DataFrame(res, columns=cursor.column_names)
df_view1

[('衣服', 1000), ('厨房用具', 3000), ('厨房用具', 500), ('办公用品', 100)]


,product_type,sale_price
0,衣服,1000
1,厨房用具,3000
2,厨房用具,500
3,办公用品,100


In [14]:
df_view

,product_type,cnt_product
0,衣服,2
1,办公用品,2
2,厨房用具,4


##### 更新视图内容
视图是一个虚拟表，对视图的操作就是对底层基础表的操作。      
视图归根结底还是从表派生出来的，因此，如果原表可以更新，那么 视图中的数据也可以更新。反之亦然，如果视图发生了改变，而原表没有进行相应更新的话，就无法保证数据的一致性了。

##### 子查询
子查询，也称为嵌套查询，是在 SQL 查询中的一个查询，它位于另一个查询的内部。子查询可以出现在不同的位置，例如 SELECT、FROM、WHERE、HAVING 等子句中，用于提供要过滤、聚合、或计算的数据。

In [18]:
### 标量子查询

# 
''' 
查询商品价格高于平均价格的商品的 product_id, product_name, sale_price
'''
subquery_ = '''SELECT product_id, product_name, sale_price
               FROM products
               WHERE sale_price > (SELECT AVG(sale_price)
                                   FROM products); '''
cursor.execute(subquery_)

res = cursor.fetchall()
print(res)

df_subquery = pd.DataFrame(res, columns=cursor.column_names)
df_subquery

[('0003', '运动T恤', 4000), ('0004', '菜刀', 3000), ('0005', '高压锅', 6800)]


,product_id,product_name,sale_price
0,0003,运动T恤,4000
1,0004,菜刀,3000
2,0005,高压锅,6800


In [19]:
subquery_1 = '''SELECT product_id, product_name, sale_price, (SELECT AVG(sale_price)
                                                              FROM products) AS avg_price
               FROM products; '''
cursor.execute(subquery_1)

res = cursor.fetchall()
print(res)

df_subquery1 = pd.DataFrame(res, columns=cursor.column_names)
df_subquery1

[('0001', 'T恤衫', 1000, Decimal('2097.5000')), ('0002', '打孔器', 500, Decimal('2097.5000')), ('0003', '运动T恤', 4000, Decimal('2097.5000')), ('0004', '菜刀', 3000, Decimal('2097.5000')), ('0005', '高压锅', 6800, Decimal('2097.5000')), ('0006', '叉子', 500, Decimal('2097.5000')), ('0007', '擦菜板', 880, Decimal('2097.5000')), ('0008', '圆珠笔', 100, Decimal('2097.5000'))]


,product_id,product_name,sale_price,avg_price
0,0001,T恤衫,1000,2097.5000
1,0002,打孔器,500,2097.5000
2,0003,运动T恤,4000,2097.5000
3,0004,菜刀,3000,2097.5000
4,0005,高压锅,6800,2097.5000
5,0006,叉子,500,2097.5000
6,0007,擦菜板,880,2097.5000
7,0008,圆珠笔,100,2097.5000


##### 关联子查询
关联子查询就是通过一些标志将内外两层的查询连接起来起到过滤数据的目的，接下来我们就一起看一下关联子查询的具体内容吧。

In [30]:
### 选取出各商品种类中高于该商品种类的平均销售单价的商品
corr_query = '''SELECT product_type, product_name, sale_price
                FROM products AS p1
                WHERE sale_price > (SELECT AVG(sale_price)
                                    FROM products AS p2
                                    WHERE p1.product_type = p2.product_type
                                    GROUP BY product_type); '''
cursor.execute(corr_query)

res = cursor.fetchall()
print(res)

df_corr_ = pd.DataFrame(res, columns=cursor.column_names)
df_corr_

[('办公用品', '打孔器', 500), ('衣服', '运动T恤', 4000), ('厨房用具', '菜刀', 3000), ('厨房用具', '高压锅', 6800)]


,product_type,product_name,sale_price
0,办公用品,打孔器,500
1,衣服,运动T恤,4000
2,厨房用具,菜刀,3000
3,厨房用具,高压锅,6800


##### 谓词
谓词是返回为真值的函数。      
谓词主要有以下几个：    
 * LIKE
 * BETWEEN
 * IS NULL, IS NOT NULL
 * IN
 * EXISTS

In [24]:
### LIKE 用于字符串的部分一致查询

# 创建表
creat_table_str = '''CREATE TABLE samplelike
                     (strcol VARCHAR(6) NOT NULL,
                      PRIMARY KEY (strcol)); '''
cursor.execute(creat_table_str)
db.commit()

insert_data_query = '''INSERT INTO samplelike (strcol) VALUES (%s)'''
data = [
    ('abcddd',),
    ('dddabc',),
    ('abdddc',),
    ('abcdd',),
    ('ddabc',),
    ('abddc',)
]

cursor.executemany(insert_data_query, data)
db.commit()

In [25]:
# LIKE
like_query = '''SELECT *
                FROM samplelike
                WHERE strcol LIKE 'ddd%'; '''

cursor.execute(like_query)

res = cursor.fetchall()
print(res)

df_data_str = pd.DataFrame(res, columns=cursor.column_names)
df_data_str

[('dddabc',)]


,strcol
0,dddabc


##### CASE 表达式
CASE 表达式在 SQL 中用于基于条件对数据进行分类换。


In [28]:
### 实现列方向上的聚合
### 搜索表达式
case_clu = '''SELECT SUM(CASE WHEN product_type = '衣服' THEN sale_price ELSE 0 END) AS sum_price_clothes,
                     SUM(CASE WHEN product_type = '厨房用具' THEN sale_price ELSE 0 END) AS sum_price_kitchen,
                     SUM(CASE WHEN product_type = '办公用品' THEN sale_price ELSE 0 END) AS sum_price_office
              FROM products; '''

cursor.execute(case_clu)
res = cursor.fetchall()

print(res)
df_case = pd.DataFrame(res, columns=cursor.column_names)
df_case

[(Decimal('5000'), Decimal('11180'), Decimal('600'))]


,sum_price_clothes,sum_price_kitchen,sum_price_office
0,5000,11180,600


In [22]:
### 3.4

# 创建视图
'''
创建视图包含4列， product_id, product_name, product_type, sale_price, sale_price_avg_type
这个视图没有限制条件，product_id, product_name, product_type, sale_price, 是直接选取的从 products 表中；
sale_price_avg_type 这列需要用到关联子查询
'''
create_view_ = '''CREATE VIEW AvgPriceByType1 (product_id, product_name, product_type, sale_price, sale_price_avg_type)
                  AS
                  SELECT product_id,
                         product_name,
                         product_type,
                         sale_price,
                         (SELECT AVG(sale_price) 
                          FROM products AS p2
                          WHERE p1.product_type = p2.product_type
                          GROUP BY p2.product_type) AS sale_price_avg_type
                    FROM products AS p1'''

cursor.execute(create_view_)
db.commit()

select_view = '''SELECT * FROM AvgPriceByType1'''
cursor.execute(select_view)

res = cursor.fetchall()
df_data2 = pd.DataFrame(res, columns=cursor.column_names)
df_data2


,product_id,product_name,product_type,sale_price,sale_price_avg_type
0,0001,T恤衫,衣服,1000,2500.0000
1,0002,打孔器,办公用品,500,300.0000
2,0003,运动T恤,衣服,4000,2500.0000
3,0004,菜刀,厨房用具,3000,2795.0000
4,0005,高压锅,厨房用具,6800,2795.0000
5,0006,叉子,厨房用具,500,2795.0000
6,0007,擦菜板,厨房用具,880,2795.0000
7,0008,圆珠笔,办公用品,100,300.0000
